In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3


In [ ]:
dir = '../input/sf-bay-area-bike-share/'
file_path = 'database.sqlite'
database = os.path.join(dir, file_path)
database 

In [ ]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""
SELECT *
FROM sqlite_master
WHERE type='table'
;""", conn)

tables

In [ ]:
trip = pd.read_sql("""
SELECT *
FROM trip
LIMIT 5
;""", conn)

trip

* How Many Stations Are Per City?

In [ ]:
station_per_city = pd.read_sql("""
SELECT DISTINCT city, COUNT(name) Stations
FROM station
GROUP BY city
;""", conn)

station_per_city

In [ ]:
trips_per_station = pd.read_sql("""
SELECT 
    s.name Station_name,
    COUNT(t.start_station_id) Starting_trips,
    COUNT(t.end_station_id) Ending_trips
FROM station s
JOIN trip t
ON s.id = t.start_station_id
AND s.id = t.end_station_id
GROUP BY 1
ORDER BY 2 DESC
;""", conn)

trips_per_station

* How Many Trips Have Been Made From Each Station?

In [ ]:
trips_from_station = pd.read_sql("""
SELECT 
    s.name Station_name,
    COUNT(t.start_station_id) Starting_trips
FROM station s
JOIN trip t
ON s.id = t.start_station_id
GROUP BY 1
ORDER BY 2 DESC
;""", conn)

trips_from_station

* How Many Trips Have Been Made To Each Station?

In [ ]:
trip_to_station = pd.read_sql("""
SELECT 
    s.name Station_Name,
    COUNT(t.end_station_id) Starting_trips
FROM station s
INNER JOIN trip t
ON s.id = t.end_station_id
GROUP BY 1
ORDER BY 2 DESC
;""", conn)

trip_to_station

* How Many Subs/Cus. Are Per Station?
* How Many Trips Are Per Station?
* How Much Percentage Does Subs/Cus. Represent In Each Station ?

In [ ]:
subs = pd.read_sql("""
SELECT 
    start_station_name,
    COUNT(CASE WHEN subscription_type='Subscriber' THEN start_station_id END) Subscribers,
    COUNT(CASE WHEN subscription_type='Customer' THEN start_station_id END) Customers,
    (SELECT COUNT(start_station_id) FROM trip t1 WHERE t.start_station_id = t1.start_station_id) Total_trips,   
    COUNT(CASE WHEN subscription_type='Subscriber' THEN start_station_id END)*100/COUNT(start_station_id) Sub_percentage,
    COUNT(CASE WHEN subscription_type='Customer' THEN start_station_id END)*100/COUNT(start_station_id) Cus_percentage
FROM trip t
GROUP BY 1
;""", conn)

subs